## Setup

In [1]:
import os
import requests
import json
from pathlib import Path

import sys
sys.path.append('../')
from modules.helpers import get_access_token, load_eoepca_state, test_cell, test_results

Load `eoepca state` environment

In [2]:
load_eoepca_state()

In [3]:
platform_domain = os.environ.get("INGRESS_HOST")
resource_health_domain = f'https://resource-health.{platform_domain}'
print(f"Resource Health URL: {resource_health_domain}")

Resource Health URL: https://resource-health.notebook-test.develop.eoepca.org


## Validate Resource Health Web Dashboard

In [4]:
web_url = f"{resource_health_domain}/"
response = requests.get(web_url)

if response.status_code == 200:
    print(f"✅ Resource Health Web dashboard accessible at: {web_url}")
else:
    print(f"❌ Dashboard access failed: {response.status_code}")

✅ Resource Health Web dashboard accessible at: https://resource-health.notebook-test.develop.eoepca.org/


## Retrieve Existing Health Checks

In [9]:
checks_url = f"{resource_health_domain}/api/healthchecks/checks/"
response = requests.get(checks_url)

checks = response.json()
for check in checks:
    print(f"- {check['id']} (Scheduled: {check['schedule']})")

- resource-health-healthchecks-cronjob-0 (Scheduled: */1 * * * *)


## Create a New Health Check via API

In [16]:
new_check_payload = {
    "template_id": "default_k8s_template",
    "template_args": {
        "script": "https://raw.githubusercontent.com/EOEPCA/resource-health/refs/tags/v0.1.0-demo/pytest-health/instrumentation/examples/mock_api_check.py",
        "requirements": "https://gist.githubusercontent.com/tilowiklundSensmetry/a9fefe2873b731b483f554607a82deaa/raw/1136a82ca3c8f28b1ad4d895871514185927dd1c/requirements.txt",
        "health_check.name": "from-notebook-automation"
    },
    "schedule": "*/1 * * * *",
    "withCredentials": True
}

response = requests.post(checks_url, json=new_check_payload)

print(f"New check created: {response.text}")

New check created: {"id":"106b3233-41c3-4961-a9c6-92d59d77f6de","metadata":{"template_id":"default_k8s_template","template_args":{"script":"https://raw.githubusercontent.com/EOEPCA/resource-health/refs/tags/v0.1.0-demo/pytest-health/instrumentation/examples/mock_api_check.py","requirements":"https://gist.githubusercontent.com/tilowiklundSensmetry/a9fefe2873b731b483f554607a82deaa/raw/1136a82ca3c8f28b1ad4d895871514185927dd1c/requirements.txt","health_check.name":"from-notebook-automation"}},"schedule":"*/1 * * * *","outcome_filter":{"resource_attributes":{"k8s.cronjob.name":"106b3233-41c3-4961-a9c6-92d59d77f6de"}}}


## Check Telemetry Service Status

In [ ]:
telemetry_url = f"{resource_health_domain}/api/telemetry/healthz"
response = requests.get(telemetry_url)

print(f"Telemetry health: {response.text}")

❌ Telemetry service check failed: 200 - "OK"
